In [3]:
import os
import glob
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
import numpy as np
import pandas as pd

/Users/ys/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/ys/.matplotlib/matplotlibrc", line #628
  (fname, cnt))


In [5]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 
           '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [6]:
batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'input/processed/train'
valid_data_dir = 'input/processed/valid'
test_data_dir = 'input/given/test'

nb_train_samples = 12399 - (15 * 24)
nb_val_samples = 15 * 24
nb_epoch = 50

result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [7]:
# keras.applications.imagenet_utilsのxは4Dテンソルなので
# 3Dテンソル版を作成
def preprocess_input(x):
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 3D.
    # Returns
        Preprocessed tensor.
    """
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [8]:
# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
# vgg16.summary()

In [9]:
# FC層を構築
# Flattenへの入力指定はバッチ数を除く
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

In [10]:
# VGG16とFCを接続
model = Model(input=vgg16.input, output=top_model(vgg16.output))

# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:15]:
    layer.trainable = False

/Users/ys/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=/input_1, outputs=Softmax.0)`
  


In [11]:
# Fine-tuningのときはSGDの方がよい？
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [12]:
# preprocessing_functionはKeras1.2.2から
# VGG16を使うための画像前処理を適用する
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 11799 images belonging to 24 classes.


In [13]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

# Fine-tuning
history = model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
    validation_data=validation_generator,
    nb_val_samples=nb_val_samples)

model.save_weights(os.path.join(result_dir, 'finetuning.h5'))

Found 360 images belonging to 24 classes.


/Users/ys/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=360, validation_data=<keras.pre..., epochs=50, steps_per_epoch=387)`


Epoch 1/50
325/387 [========================>.....] - ETA: 567s - loss: 3.7159 - acc: 0.1583

KeyboardInterrupt: 

In [ ]:
# load test data and make prediction
path = os.path.join('input', 'given', 'test', '*.jpg')
files = sorted(glob.glob(path))

In [ ]:
X_test = []
X_test_id = []
    
for fl in files:
    flbase = os.path.basename(fl)
    img = image.load_img(fl, target_size=(img_rows, img_cols))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    X_test.append(model.predict(x))
    X_test_id.append(flbase)

df = pd.DataFrame.from_records(X_test, index=X_test_id)
df.to_csv('sub.csv')